In [2]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import TypedDict
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# model
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)

In [4]:
# parser
parser = StrOutputParser()

In [5]:
# create a state

class QNAState(TypedDict):

    question: str
    ans: str

In [9]:
def llm_qa(state : QNAState) -> QNAState:
      
      # extracting question from the state
      question = state['question']
      
      # form a prompt template
      template = PromptTemplate.from_template(
          template = "Answer the following question : {question}"
      )
      
      # chain
      chain = template | model | parser
      
      # ask the question from the llm
      answer = chain.invoke({"question" : question})
      
      # update the answer from the state
      state['ans'] = answer
      
      return state

In [10]:
# create our graph

graph = StateGraph(QNAState)

# add nodes
graph.add_node('llm_qa', llm_qa)

# add edges
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

# compile
workflow = graph.compile()

In [11]:
# execute

intial_state = {'question': 'How far is moon from the earth?'}

final_state = workflow.invoke(intial_state)

In [18]:
final_state

{'question': 'How far is moon from the earth?',
 'ans': "The average distance from the Earth to the Moon is approximately **384,400 kilometers (238,900 miles)**.\n\nHowever, this distance is not constant because the Moon's orbit around the Earth is elliptical, not perfectly circular. Therefore, the distance varies:\n\n*   **At its closest point (perigee):** Approximately 363,104 km (225,623 miles)\n*   **At its farthest point (apogee):** Approximately 406,696 km (252,088 miles)\n\nSo, while 384,400 km is the widely cited average, it's important to remember it fluctuates throughout the month."}

In [16]:
final_state['ans']

"The average distance from the Earth to the Moon is approximately **384,400 kilometers (238,900 miles)**.\n\nHowever, this distance is not constant because the Moon's orbit around the Earth is elliptical, not perfectly circular. Therefore, the distance varies:\n\n*   **At its closest point (perigee):** Approximately 363,104 km (225,623 miles)\n*   **At its farthest point (apogee):** Approximately 406,696 km (252,088 miles)\n\nSo, while 384,400 km is the widely cited average, it's important to remember it fluctuates throughout the month."